In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from qiskit import QuantumCircuit
from qiskit import QuantumRegister
from qiskit import ClassicalRegister
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 
import numpy as np
import random

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol without an attacker.
root2 = math.sqrt(2)
root3 = math.sqrt(3)


denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 

B0_transform_matrix = [ [ -1 / denom1 , (1 + root2) / denom1 ],
                        [  1 / denom2 , (root2 - 1) / denom2 ] ]
B0_transform = Operator(B0_transform_matrix) 


B1_transform_matrix = [ [  1 / denom1 , (1 + root2) / denom1 ],
                        [ -1 / denom2 , (root2 - 1) / denom2 ] ]
B1_transform = Operator(B1_transform_matrix) 

A2_transform_matrix = B0_transform_matrix
A1_transform = Operator(A2_transform_matrix)

# Unitary Operator for probability
unitary_op = [ [ 1/root3      , -(root2/root3) ],
               [ root2/root3  , 1/root3        ] ]


# Function to create entangled pair of qubits
def entangledPair():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    return qc

def average(c,n): 
    backend = BasicSimulator()
    compiled = transpile(c, backend)
    job_sim = backend.run(compiled, shots=n)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)
    # print(counts)
    count00 = counts.get("00",0) 
    count01 = counts.get("01",0) 
    count10 = counts.get("10",0) 
    count11 = counts.get("11",0) 
    # The return value includes the conversion from measurement results 0,1 to +1,-1
    # Each 00 means a value of  1 (+1 * +1)
    # Each 01 means a value of -1 (+1 * -1)
    # Each 10 means a value of -1 (-1 * +1)
    # Each 11 means a value of  1 (+1 * +1)
    return (count00 - count01 - count10 + count11) / n 

def random_basis(user):
    qc = QuantumCircuit(2, 2)
    qc.unitary(unitary_op, [0])
    qc.h(1)
    
    qc.measure([0, 1], [0, 1])

    backend = BasicSimulator()
    compiled = transpile(qc, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result() 
    counts = result_sim.get_counts(compiled)

    measured_bits = list(counts.keys())[0]

    if user == "Alice":
        if measured_bits[0] == '0':
            return "X"
        else:
            return "W" if measured_bits[1] == '0' else "Z"

    if user == "Bob":
        if measured_bits[0] == '0':
            return "W"
        else:
            return "Z" if measured_bits[1] == '0' else "V"

# def simulate_single_run():
#     qc = entangledPair()

#     # alice_basis = random.choice(["X", "W", "Z"])
#     # bob_basis = random.choice(["W", "Z", "V"])

#     alice_basis = random_basis("Alice")
#     bob_basis = random_basis("Bob")
    
#     if alice_basis == "X":
#         qc.h(0)
#     elif alice_basis == "W":
#         qc.unitary(A2_transform_matrix, [0])

#     if bob_basis == "W":
#         qc.unitary(B0_transform_matrix, [1])
#     elif bob_basis == "V":
#         qc.unitary(B1_transform_matrix, [1])
    
#     qc.measure_all()
#     return qc, alice_basis, bob_basis

def eve_intercept(qc):
    eve_basis = random_basis("Alice")

    if eve_basis == "X":
        qc.h(0)
    elif eve_basis == "W":
        qc.unitary(A2_transform_matrix, [0])

    # if eve_basis == "X":
    #     qc.h(0)
    # elif eve_basis == "W":
    #     qc.unitary(A2_transform_matrix, [0])

    return eve_basis

def simulate_single_run_eve():
    qc = entangledPair()
    eve_basis = eve_intercept(qc)
    
    alice_basis = random_basis("Alice")
    bob_basis = random_basis("Bob")

    if alice_basis == "X":
        qc.h(0)
    elif alice_basis == "W":
        qc.unitary(A2_transform_matrix, [0])

    if bob_basis == "W":
        qc.unitary(B0_transform_matrix, [1])
    elif bob_basis == "V":
        qc.unitary(B1_transform_matrix, [1])

    qc.measure_all()
    return qc, alice_basis, bob_basis, eve_basis

def run_ekert91_intercept(N):
    matches = 0
    total_key_size = 0
    total_measurements = 0
    interceptor_detected = 0
    shared_key = []
    
    averages = {
        "XW": 0, "XW_avg": 0, "XV": 0, "XV_avg": 0,
        "ZW": 0, "ZW_avg": 0, "ZV": 0, "ZV_avg": 0
    }
    
    for _ in range(N):
        # qc, alice_basis, bob_basis = simulate_single_run()
        qc, alice_basis, bob_basis, eve_basis = simulate_single_run_eve()
        # print(qc, alice_basis, bob_basis)
        # print(qc, alice_basis, bob_basis, eve_basis)
        
        avg = average(qc, 100)
        # print(avg, alice_basis, bob_basis, eve_basis)

        if (alice_basis != bob_basis) and avg == 1:
            interceptor_detected += 1
        
        if (alice_basis == "Z" and bob_basis == "Z") or (alice_basis == "W" and bob_basis == "W"):
            total_key_size += 1
            matches += (1 if avg == 1 else 0)
            shared_key.append(alice_basis)

        if (alice_basis, bob_basis) in [("X", "W"), ("X", "V"), ("Z", "W"), ("Z", "V")]:
            total_measurements += 1
            if (alice_basis, bob_basis) == ("X", "W"):
                averages["XW"] += 1
                averages["XW_avg"] += avg
            elif (alice_basis, bob_basis) == ("X", "V"):
                averages["XV"] += 1
                averages["XV_avg"] -= avg
            elif (alice_basis, bob_basis) == ("Z", "W"):
                averages["ZW"] += 1
                averages["ZW_avg"] += avg
            elif (alice_basis, bob_basis) == ("Z", "V"):
                averages["ZV"] += 1
                averages["ZV_avg"] += avg

    xw_total = averages["XW_avg"]/averages["XW"]
    xv_total = averages["XV_avg"]/averages["XV"]
    zw_total = averages["ZW_avg"]/averages["ZW"]
    zv_total = averages["ZV_avg"]/averages["ZV"]

    S = abs(xv_total + xv_total + zw_total + zv_total)
    return round(S, 3)

N = 100
print(run_ekert91_intercept(N))

0.029
